# Assignment 5 - Ensemble Learning

We have seen how to create simple classifiers. But some problems are beyond the abilities of these models, so we have two options:

Build more complicated learners using sophisticated methods.
Use more simple classifiers.
We are going to opt for the second approach.

For this assignment, we want to create ensemble learners, which are based on the simple models we have seen so far. Much like the "wisdom of the crowds", we wish to exploit the wisdom of multiple classifiers to make better predictions than they would individually.

We will explore two main classes of ensemble learners: bagging and boosting.

In [ ]:
import hashlib
import numpy as np

def pseudo_random_x(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed)/0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits)/0xffffffff
        yield r


def bootstrap_x(dataset, sample_size):
    rand = pseudo_random()
    while True:
        result = []
        for i in range(sample_size):
            r = next(rand)
            index = int(r * len(dataset))
            result.append(dataset[index])
        yield np.array(result)


def voting_ensemble_x(classifiers):
    def f(point):
        d = {}
        for c in classifiers:
            d[c(point)] = d.get(c(point), 0) + 1
        invert = [(y, -x) for x, y in d.items()]
        return -max(invert)[1]
    return f

def bagging_model_x(learner, dataset, n_models, sample_size):
    type(dataset)
    models = []
    samples = bootstrap(dataset, sample_size)
    for i in range(n_models): # for each of the t iterations
        sample = next(samples)  # sample n instances from training set
        model = learner(sample)
        models.append(model)
    return voting_ensemble(models)


class weighted_bootstrap:
    def init(self, dataset, weights, sample_size):
        self.dataset = dataset
        self.weights = weights
        self.sample_size = sample_size
        self.rand = pseudorandom()

    def iter(self):
        return self

    def next(self):
        sample = []
        for  in range(self.sample_size):
            index = self.weighted_sample()
            sample.append(self.dataset[index])
        return np.array(sample)

    def weighted_sample(self):
        running_sum = list(accumulate(self.weights))
        total = running_sum[-1]
        r = next(self.rand) * total
        index = bisect_right(running_sum, r)
        return index



############### Q5

import hashlib
from itertools import accumulate
from bisect import bisect_right
import numpy as np
import math


def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed) / 0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits) / 0xffffffff
        yield r


class weighted_bootstrap:
    def __init__(self, dataset, weights, sample_size):
        self.dataset = dataset
        self.weights = weights
        self.sample_size = sample_size
        self.rand = pseudo_random()

    def __iter__(self):
        return self

    def __next__(self):
        sample = []
        for _ in range(self.sample_size):
            index = self.weighted_sample()
            sample.append(self.dataset[index])
        return np.array(sample)

    def weighted_sample(self):
        running_sum = list(accumulate(self.weights))
        total = running_sum[-1]
        r = next(self.rand) * total
        index = bisect_right(running_sum, r)
        return index


def adaboost(learner, dataset, n_models):
    models = []
    weights = [1/dataset.shape[0]] * dataset.shape[0] # assign equal weight to each training instance
    classes = set()
    samples = weighted_bootstrap(dataset, weights, dataset.shape[0] )
    for t in range(n_models): # for t iterations
        sample = next(samples) # get next weighted dataset
        model = learner(sample)    # Apply learning algorithm to weighted dataset
        e = 0
        for i in range(len(dataset)):
            prediction = int(model(dataset[i][:-1]))
            target = int(dataset[i][-1])
            misclassified = 1 if target != prediction else 0
            e += misclassified * weights[i]
        if e == 0 or e >= 0.5:
            break
        for i in range(len(dataset)):
            prediction = int(model(dataset[i][:-1]))
            target = int(dataset[i][-1])
            classes.add(target)
            misclassified = 1 if target != prediction else 0
            if misclassified == 0:
                weights[i] *= e/(1-e)
        models.append((model, e))
        weights = [w/sum(weights) for w in weights]
        samples.weights = weights

    def boosted_model(data, models=models):
        num_class = len(classes)
        class_weights = [0] * num_class
        for model, e in models:
            prediction = int(model(data))
            if e == 0:
                class_weights[prediction] +=  float('inf')
            else:
                class_weights[prediction] += -1 * math.log(e / (1- e))
        return class_weights.index(max(class_weights))

    return boosted_model





import sklearn.datasets
import sklearn.utils
import sklearn.linear_model

digits = sklearn.datasets.load_digits()
data, target = sklearn.utils.shuffle(digits.data, digits.target, random_state=3)
train_data, train_target = data[:-5, :], target[:-5]
test_data, test_target = data[-5:, :], target[-5:]
dataset = np.hstack((train_data, train_target.reshape((-1, 1))))

def linear_learner(dataset):
    features, target = dataset[:, :-1], dataset[:, -1]
    model = sklearn.linear_model.SGDClassifier(random_state=1, max_iter=1000, tol=0.001).fit(features, target)
    return lambda v: model.predict(np.array([v]))[0]

boosted = adaboost(linear_learner, dataset, 10)
for (v, c) in zip(test_data, test_target):
    print(int(boosted(v)), c)

import sklearn.datasets
import sklearn.utils
import sklearn.linear_model

iris = sklearn.datasets.load_iris()
data, target = sklearn.utils.shuffle(iris.data, iris.target, random_state=0)
train_data, train_target = data[:-5, :], target[:-5]
test_data, test_target = data[-5:, :], target[-5:]
dataset = np.hstack((train_data, train_target.reshape((-1, 1))))

def linear_learner(dataset):
    features, target = dataset[:, :-1], dataset[:, -1]
    model = sklearn.linear_model.SGDClassifier(random_state=1, max_iter=1000, tol=0.001).fit(features, target)
    return lambda v: model.predict(np.array([v]))[0]

boosted = adaboost(linear_learner, dataset, 10)
for (v, c) in zip(test_data, test_target):
    print(int(boosted(v)), c)

## Q1

Define a function voting_ensemble(classifiers) that takes as a parameter a list of classifiers, and returns a new classifier that reports, for a given input, the winning vote amongst all the given classifiers on that input.

You can assume that the input that is passed onto the output of voting_ensemble can be consumed by all the classifiers in classifiers.

In the case of a tie, return the output that sorts lowest (whether this is numeric or lexicographic). You may assume there will be at least one classifier.

In [ ]:
def voting_ensemble(classifiers):
  def f(data):
    dicty = {}
    for classifier in classifiers:
      dicty[classifier(data)] = dicty.get(classifier(data), 0) + 1
    return -1 * max([(y, -x) for x, y in dicty.items()])[1]
  return f

# def voting_ensemble(classifiers):
#   def f(data):
#     dicty = {}
#     for classifier in classifiers:
#       dicty[classifier(data)] = dicty.get(classifier(data), 0) + 1
#     return max(dicty, key=dicty.get)
#   return f

# Modelling y > x^2
classifiers = [
    lambda p: 1 if 1.0 * p[0] < p[1] else 0,
    lambda p: 1 if 0.9 * p[0] < p[1] else 0,
    lambda p: 1 if 0.8 * p[0] < p[1] else 0,
    lambda p: 1 if 0.7 * p[0] < p[1] else 0,
    lambda p: 1 if 0.5 * p[0] < p[1] else 0,
]
data_points = [(0.2, 0.03), (0.1, 0.12), 
               (0.8, 0.63), (0.9, 0.82)]
c = voting_ensemble(classifiers)
for v in data_points:
    print(c(v))

print("Result should match 0 1 0 1")

0
1
0
1
Result should match 0 1 0 1


In [ ]:
#hidden test
classifiers = [lambda x: 0, lambda x: 1]
c1 = voting_ensemble(classifiers)
c2 = voting_ensemble(classifiers[::-1])
print(c1("Hello"))
print(c2("Goodbye"))

print("Result should match 0 0")

0
0
Result should match 0 0


# Q2


Define a function bootstrap(dataset, sample_size) that samples the given dataset to produce samples of sample_size. Your function should be a generator: that is, calling it produces an iterator itr that produces a new sample when next(itr) is called.

Datasets are given as numpy arrays, where each row is a single feature vector. You should sample the rows. Use the following algorithm: use our pseudo_random to generate a random number r, convert it to an index by int(r * len(dataset)), then add the row at that index to the sample. Once the sample has sample_size many rows, yield the sample.

Note that by using yield your code automatically becomes a generator.

Python provides a wonderful random number generator in the random module. Unfortunately, even if we seed it, there is no guarantee that we will get the same sequence of random numbers across all versions and operating systems. While it will likely be the same, to make things easier for you (and us), we provide the following code to generate consistent random numbers everywhere. 

import hashlib
def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed)/0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits)/0xffffffff
        yield r

def take(n, iterator):
    while n > 0:
        yield next(iterator)
        n -= 1

for i in take(5, pseudo_random()):
    print(i)
This is not a great random number generator, but it's not as terrible as it first seems: using hashes as a source for random numbers is possible. You are welcome to research this in your own time.

We will use this pseudo_random function for most of the remaining questions.

In [ ]:
import hashlib
import numpy as np

def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed)/0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits)/0xffffffff
        yield r

def bootstrap(dataset, sample_size):
  randy = pseudo_random()
  while randy:
    results = []
    for _ in range(sample_size):
      results.append(dataset[int(next(randy) * len(dataset))])
    yield np.array(results)


dataset = np.array([[1, 0, 2, 3],
                    [2, 3, 0, 0],
                    [4, 1, 2, 0],
                    [3, 2, 1, 0]])
ds_gen = bootstrap(dataset, 3)
print(next(ds_gen))
print(next(ds_gen))

print(f"match [[4 1 2 0] [2 3 0 0] [1 0 2 3]][[3 2 1 0] [2 3 0 0] [1 0 2 3]]")

[[4 1 2 0]
 [2 3 0 0]
 [1 0 2 3]]
[[3 2 1 0]
 [2 3 0 0]
 [1 0 2 3]]
match [[4 1 2 0] [2 3 0 0] [1 0 2 3]][[3 2 1 0] [2 3 0 0] [1 0 2 3]]


# Q3


Define a function bagging_model(learner, dataset, n_models, sample_size) that returns a new model based on the learner, but replicated n_models times over the bootstrapped dataset of size sample_size.

A model is a function that takes a feature vector (a d length numpy array) and produces a prediction (value).

A learner is a function that takes a dataset and produces a model.

The dataset is a n×(d+1) numpy array where each row is a new feature vector; the final column is the class. 

Use your voting_ensemble and bootstrap functions. Don't forget to include them, along with their dependencies – including the random number generator!

Hint: If your function is more than half a dozen lines, you might be going about this the wrong way.

For example:

In [ ]:
import hashlib
import numpy as np

def voting_ensemble(classifiers):
  def f(data):
    dicty = {}
    for classifier in classifiers:
      dicty[classifier(data)] = dicty.get(classifier(data), 0) + 1
    return -1 * max([(y, -x) for x, y in dicty.items()])[1]
  return f

def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed)/0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits)/0xffffffff
        yield r

def bootstrap(dataset, sample_size):
  randy = pseudo_random()
  while randy:
    results = []
    for _ in range(sample_size):
      results.append(dataset[int(next(randy) * len(dataset))])
    yield np.array(results)

def bagging_modelx(learner, dataset, n_models, sample_size):
    type(dataset)
    models = []
    samples = bootstrap(dataset, sample_size)
    for i in range(n_models): # for each of the t iterations
        sample = next(samples)  # sample n instances from training set
        model = learner(sample)
        models.append(model)
    return voting_ensemble(models)

def bagging_model(learner, dataset, n_models, sample_size):
  m = []
  s = bootstrap(dataset, sample_size)
  for _ in range(n_models):
    m.append(learner(next(s)))
  return voting_ensemble(m)

import sklearn.datasets
import sklearn.utils
import sklearn.tree

iris = sklearn.datasets.load_iris()
data, target = sklearn.utils.shuffle(iris.data, iris.target, random_state=1)
train_data, train_target = data[:-5, :], target[:-5]
test_data, test_target = data[-5:, :], target[-5:]
dataset = np.hstack((train_data, train_target.reshape((-1, 1))))

def tree_learner(dataset):
    features, target = dataset[:, :-1], dataset[:, -1]
    model = sklearn.tree.DecisionTreeClassifier(random_state=1).fit(features, target)
    return lambda v: model.predict(np.array([v]))[0]

bagged = bagging_model(tree_learner, dataset, 50, len(dataset)//2)
# Note that we get the first one wrong!
for (v, c) in zip(test_data, test_target):
    print(int(bagged(v)), c)

print("\nshould match below\n1 2\n2 2\n1 1 \n2 2 \n0 0")

1 2
2 2
1 1
2 2
0 0

should match below
1 2
2 2
1 1 
2 2 
0 0


# Q4

Bagging works well, but it doesn't consider how good any of the previous models were at classifying the data. Instead, we can consider how good the existing models are, and specifically train the next models to fill in their shortcomings. This is called boosting.

We will implement an algorithm called AdaBoost. This algorithm works by weighting the likelihood of sampling different values from the dataset based on how often they are incorrectly classified. Then, when making a prediction, the final class is no longer just a vote from the models: the error of each model is factored in the vote. Thus we combine a series of models that "adapt" to the "bootstraps" of the dataset.




Define a class weighted_bootstrap that is initialised with a dataset, weights, and a sample_size, where dataset and weights must be the same length. This class should be an iterator (that is, __iter__ should return self), and each call to next (that is, the __next__ method) should produce a new bootstrapped sample of sample_size rows, where each row has a chance of occurring proportional to its weight. 

The class should allow modification of the weights attribute so that the weights can change dynamically.

The algorithm we use to do this weighted sample is as follows:

Calculate the running sum of the weights
Generate a random value up to the sum of the weights
Find the index i of the first value in the running sum to exceed this random value.
The row in position i in the dataset gets added to the sample.
Repeat until a complete sample is drawn. 
Notes
Don't forget to use our pseudo-random number generator.
We are violating the naming convention that classes are CamelCase to make weighted_bootstrap seem like a function, because we want it to be used like a function. This seems a bit icky, but there is precedent... have you ever looked closely at the built-in range "function"?
The Airfoil Self-Noise Data used in some test cases is available here
The running sum of [1, 2, 3] is [1, 3, 6].
Pay attention to the sentence "Generate a random value up to the sum of the weights." The sum of weights is not necessarily an integer!

In [ ]:
import hashlib
from itertools import accumulate
from bisect import bisect_right
import numpy as np
import math


def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed) / 0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits) / 0xffffffff
        yield r


class weighted_bootstrap:
    def __init__(self, dataset, weights, sample_size):
        self.dataset = dataset
        self.weights = weights
        self.sample_size = sample_size
        self.randy = pseudo_random()

    def __next__(self):
        results = []
        for _ in range(self.sample_size):
            results.append(self.dataset[self.weighted_sample()])
        return np.array(results)

    def __iter__(self):
        return self

    def weighted_sample(self):
        running_sum = list(accumulate(self.weights))
        result = bisect_right(running_sum, next(self.randy) * running_sum[-1])
        return result



wbs = weighted_bootstrap([1, 2, 3, 4, 5], [1, 1, 1, 1, 1], 5)
sample = next(wbs)
print(type(sample))
print(sample)

print(next(wbs))
print()
wbs.weights = [1, 1, 1000, 1, 1]
print(next(wbs))
print(next(wbs))



print("\n\nShould match following:\n<class 'numpy.ndarray'> \n[3 2 1 5 3] \n[1 3 2 1 3] \n\n[3 3 3 3 3] \n[3 3 3 3 3]")

<class 'numpy.ndarray'>
[3 2 1 5 3]
[1 3 2 1 3]

[3 3 3 3 3]
[3 3 3 3 3]


Should match following:
<class 'numpy.ndarray'> 
[3 2 1 5 3] 
[1 3 2 1 3] 

[3 3 3 3 3] 
[3 3 3 3 3]


In [ ]:
# hidden test
	
wbs = weighted_bootstrap([1, 2, 3, 4, 5], [1, 2, 1, 2, 1], 5)
print(next(wbs))
print(next(wbs))
print()
wbs.weights = [5, 2, 1, 4, 2]
print(next(wbs))
print(next(wbs))
print()
wbs.weights = [0, 0, 1, 0, 1]
print(next(wbs))
print(next(wbs))

print("\n\nShould match following:\n[4 2 1 5 3] \n[2 3 2 1 3] \n\n[5 5 4 1 4]\n[1 4 1 4 2]\n\n[3 5 5 3 3]\n[5 3 5 3 3]")

[4 2 1 5 3]
[2 3 2 1 3]

[5 5 4 1 4]
[1 4 1 4 2]

[3 5 5 3 3]
[5 3 5 3 3]


Should match following:
[4 2 1 5 3] 
[2 3 2 1 3] 

[5 5 4 1 4]
[1 4 1 4 2]

[3 5 5 3 3]
[5 3 5 3 3]


# Q5


Define a function adaboost(learner, dataset, n_models) that builds a boosted ensemble model consisting of n_models simple models made from the learner, which were trained on weighted bootstrapped samples from the dataset.

As with bagging, a learner is a function that takes a dataset and returns a model. A model is a function which takes a feature vector and returns a classification.

Follow the pseudocode in the lecture notes.

Notes
Remember to include all the code from your support functions, including the weighted bootstrapping class and the random number generator.
Boosting is more sophisticated that bagging, half a dozen lines won't cut it here.
To compute the model's error on the dataset, add up the weight of the instances (rows) that are misclassified. This error is used to update the weights. It is also stored with the model so that later, when combining the outputs of the classifiers, it can be used to compute the correct weight for the classifier.
When e=0, define the value of log(e/(1-e)) to be -infinity.

In [ ]:
import hashlib
from itertools import accumulate
from bisect import bisect_right
import numpy as np
import math


def pseudo_random(seed=0xDEADBEEF):
    """Generate an infinite stream of pseudo-random numbers"""
    state = (0xffffffff & seed) / 0xffffffff
    while True:
        h = hashlib.sha256()
        h.update(bytes(str(state), encoding='utf8'))
        bits = int.from_bytes(h.digest()[-8:], 'big')
        state = bits >> 32
        r = (0xffffffff & bits) / 0xffffffff
        yield r


class weighted_bootstrap:
    def __init__(self, dataset, weights, sample_size):
        self.dataset = dataset
        self.weights = weights
        self.sample_size = sample_size
        self.randy = pseudo_random()

    def __next__(self):
        results = []
        for _ in range(self.sample_size):
            results.append(self.dataset[self.weighted_sample()])
        return np.array(results)

    def __iter__(self):
        return self

    def weighted_sample(self):
        running_sum = list(accumulate(self.weights))
        result = bisect_right(running_sum, next(self.randy) * running_sum[-1])
        return result


def adaboost(learner, dataset, n_models):
    models, class_set = [], set()
    weights = [1/dataset.shape[0]] * dataset.shape[0]
    w_bs = weighted_bootstrap(dataset, weights, dataset.shape[0])
    for t in range(n_models):
        e, m = 0, learner(next(w_bs))
        for i in range(len(dataset)):
            mis_clas = 1 if int(dataset[i][-1]) != int(m(dataset[i][:-1])) else 0
            e += mis_clas * weights[i]
        if e >= 0.5 or e == 0:
            break
        for i in range(len(dataset)):
            class_set.add(int(dataset[i][-1]))
            mis_clas = 1 if int(dataset[i][-1]) != int(m(dataset[i][:-1])) else 0
            if mis_clas == 0:
                weights[i] *= e/(1-e)
        models.append((m, e))
        weights = [weight/sum(weights) for weight in weights]
        w_bs.weights = weights

    def boosted_model(data, models=models):
        num_class = len(class_set)
        weights = [0] * num_class
        for m, e in models:
            p = int(m(data))
            if e != 0:
                weights[p] += -1 * math.log(e / (1- e))
            else:
                weights[p] = float('inf')
        result = weights.index(max(weights))
        return result

    return boosted_model







import sklearn.datasets
import sklearn.utils
import sklearn.linear_model

digits = sklearn.datasets.load_digits()
data, target = sklearn.utils.shuffle(digits.data, digits.target, random_state=3)
train_data, train_target = data[:-5, :], target[:-5]
test_data, test_target = data[-5:, :], target[-5:]
dataset = np.hstack((train_data, train_target.reshape((-1, 1))))

def linear_learner(dataset):
    features, target = dataset[:, :-1], dataset[:, -1]
    model = sklearn.linear_model.SGDClassifier(random_state=1, max_iter=1000, tol=0.001).fit(features, target)
    return lambda v: model.predict(np.array([v]))[0]

boosted = adaboost(linear_learner, dataset, 10)
for (v, c) in zip(test_data, test_target):
    print(int(boosted(v)), c)

print("\n\nShould match: \n6 6\n1 1\n0 0\n2 2\n1 1")

6 6
1 1
0 0
2 2
1 1


Should match: 
6 6
1 1
0 0
2 2
1 1
